In [3]:

import os
import pandas as pd
from PIL import Image
from dotenv import load_dotenv
from google.oauth2 import service_account
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request

In [4]:
# Load environment variables from .env file
load_dotenv()

key_file_name = os.getenv('KEY_FILE_NAME')
folder_id = os.getenv('MY_FOLDER_ID')

In [5]:
# It controls the access of the application. Currently it allows read and write both
scope = ['https://www.googleapis.com/auth/drive.readonly']

current_directory = os.getcwd()
service_account_json_key = os.path.join(current_directory, key_file_name)

credentials = service_account.Credentials.from_service_account_file(
                              filename=service_account_json_key, 
                              scopes=scope)

service = build('drive', 'v3', credentials=credentials)

In [6]:
def get_image_dimensions(image_path):
    # Open the image and get its dimensions
    with Image.open(image_path) as img:
        return img.size

In [ ]:
page_token = None

# Google drive can serve upto 1000 files in one request, page solves this issue by accessing the next in the order
while True:
    results = service.files().list(
        pageSize=1000,  
        fields="nextPageToken, files(id, name)",
        q=f"'{folder_id}' in parents",
        pageToken=page_token
    ).execute()

    files = results.get('files', [])
    
    if not files:
        print(f'No more files found in folder with ID: {folder_id}')
        break

    # Print the name of files in the folder
    for file in files:
        file_id = file['id']
        file_name = file['name']
        request = service.files().get_media(fileId=file_id)
        fh = open(file_name, 'wb')
        media_request = request.execute()
        fh.write(media_request)
        fh.close()
        dimensions = get_image_dimensions(file_name)
        print(f"Image: {file_name}, Dimensions: {dimensions}")

    
    # Get the next page token
    page_token = results.get('nextPageToken')

    if not page_token:
        break